In [1]:
# Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# Data science
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# ML Helpers/Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import unicodedata
import re

In [2]:
# Read in the 'games' dataset.
df_games = pd.read_csv("data/games.csv")
df_games.sort_values(by="name", ascending=True, ignore_index=True).head()

,id,type,name,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,users_rated,average_rating,bayes_average_rating,total_owners,total_traders,total_wanters,total_wishers,total_comments,total_weights,average_weight
0,42039,boardgame,,2004.0,2.0,4.0,40.0,40.0,40.0,10.0,10,6.0,0.0,11,0,1,5,1,0,0.0
1,182991,boardgame,,1979.0,2.0,4.0,60.0,45.0,60.0,5.0,0,0.0,0.0,0,0,0,0,0,0,0.0
2,107551,boardgame,,2010.0,2.0,2.0,60.0,60.0,60.0,12.0,3,7.0,0.0,5,0,2,2,1,0,0.0
3,87749,boardgame,,2006.0,2.0,2.0,60.0,60.0,60.0,7.0,1,6.0,0.0,0,0,1,0,0,0,0.0
4,180080,boardgame,,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1,0,0,0,0,0,0.0


In [3]:
# Drop duplicated rows.
df_games.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df_games.shape

(79463, 20)

In [4]:
# Read in the 'top 5000' dataset.
df_top5000 = pd.read_csv("data/top-5000-20190206.csv")
df_top5000.sort_values(by="names", ascending=True, ignore_index=True).head()

,rank,bgg_url,game_id,names,min_players,max_players,avg_time,min_time,max_time,year,avg_rating,geek_rating,num_votes,image_url,thumb_url,age,mechanic,owned,category,designer,publisher,weight,expands,reimplements,num_fans
0,2745,https://boardgamegeek.com/boardgame/153999/and...,153999,"...and then, we held hands.",2,2,45,30,45,2015,6.18122,5.88182,1590,https://cf.geekdo-images.com/original/img/bpIX...,https://cf.geekdo-images.com/thumb/img/myTPnQ4...,12,"Cooperative Play, Hand Management, Point to Po...",3503,"Card Game, Print & Play","David Chircop, Yannick Massa",LudiCreations,1.7400,NaN,NaN,113
1,4406,https://boardgamegeek.com/boardgame/853/und-ts...,853,...und tschüss!,4,6,30,30,30,1997,6.42761,5.68766,305,https://cf.geekdo-images.com/original/img/M7UC...,https://cf.geekdo-images.com/thumb/img/hTBB5cJ...,10,Simultaneous Action Selection,387,Card Game,Martin Wallace,"Artra Design, Ltd.",1.3000,NaN,NaN,3
2,1764,https://boardgamegeek.com/boardgame/7865/10-da...,7865,10 Days in Africa,2,4,30,20,30,2003,6.56139,6.11847,1827,https://cf.geekdo-images.com/original/img/k0YH...,https://cf.geekdo-images.com/thumb/img/Kk309Ut...,10,"Hand Management, Route/Network Building, Tile ...",2087,"Educational, Travel","Alan R. Moon, Aaron Weissblum",Beautiful Africa,1.3879,NaN,NaN,22
3,1975,https://boardgamegeek.com/boardgame/22398/10-d...,22398,10 Days in Asia,2,4,25,25,25,2007,6.68566,6.05385,1070,https://cf.geekdo-images.com/original/img/MVy_...,https://cf.geekdo-images.com/thumb/img/wXw5vxT...,10,"Hand Management, Route/Network Building, Tile ...",1303,"Educational, Travel","Alan R. Moon, Aaron Weissblum",Out of the Box Publishing,1.4286,NaN,NaN,15
4,1740,https://boardgamegeek.com/boardgame/5867/10-da...,5867,10 Days in Europe,2,4,30,30,30,2002,6.60426,6.12848,1709,https://cf.geekdo-images.com/original/img/ArgZ...,https://cf.geekdo-images.com/thumb/img/KceoH7G...,10,"Hand Management, Route/Network Building",2048,"Educational, Travel","Grafik Studio Krüger, Alan R. Moon, Aaron Weis...",Out of the Box Publishing,1.3452,NaN,NaN,24


In [5]:
# Rename 'top 5000' columns to sync with matching 'games' columns.
df_top5000 = df_top5000.rename(columns={"game_id": "id",
                           "names": "name",
                           "min_players": "minplayers",
                           "max_players": "maxplayers",
                           "avg_time": "playingtime",
                           "min_time": "minplaytime",
                           "max_time": "maxplaytime",
                           "year": "yearpublished",
                           "avg_rating": "average_rating",
                           "num_votes": "users_rated",
                           "owned": "total_owners",
                           "weight": "total_weights"})
df_top5000.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639


In [6]:
# Look at publisher counts.
df_top5000.publisher.value_counts().tail(20)

Mystics.nl                    1
Underground Games Inc. (I)    1
Cappelen                      1
Bink Ink LLC                  1
Lay Waste Games               1
Beacon Multimedia S.A.        1
Titan Forge Games             1
Crowd Games                   1
Blacklist Games               1
Ericsson                      1
Grimpeur                      1
Mortensen Games               1
Dark Ice Games                1
Copag Cards                   1
Innovention Toys              1
Frosted Games                 1
Rightgames RBG SIA            1
Dagoy                         1
Jon Sudbury Games             1
Diset S. A.                   1
Name: publisher, dtype: int64

In [7]:
# Look at designer counts.
df_top5000.designer.value_counts()

Reiner Knizia                               114
(Uncredited)                                 61
Martin Wallace                               51
Klaus Teuber                                 37
none                                         35
                                           ... 
Uwe Eickert, Gunter Eickert, Dean Halley      1
Michael Eskue                                 1
Chris Darden                                  1
Eric M. Lang, Kevin Wilson                    1
Jon Perry, Derek Yu                           1
Name: designer, Length: 2483, dtype: int64

In [8]:
# Clean and split mechanic, category, and designer columns.
df_top5000['mechanic_count'] = df_top5000.mechanic.apply(lambda x: len(x.split(",")))
df_top5000['mechanic_clean'] = df_top5000.mechanic.apply(lambda x: x.split(",")[0])
df_top5000['category_count'] = df_top5000.category.apply(lambda x: len(x.split(",")))
df_top5000['category_clean'] = df_top5000.category.apply(lambda x: x.split(",")[0])
df_top5000['designer_count'] = df_top5000.designer.apply(lambda x: len(x.split(",")))
df_top5000['designer_clean'] = df_top5000.designer.apply(lambda x: x.split(",")[0])
df_top5000.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta


In [9]:
# Look at new mechanic_clean counts.
df_top5000.mechanic_clean.value_counts()

Dice Rolling                     644
Area Control / Area Influence    566
Hand Management                  479
Card Drafting                    452
Action Point Allowance System    393
Cooperative Play                 285
Auction/Bidding                  278
Area Movement                    155
Action / Movement Programming    143
Hex-and-Counter                  138
none                             137
Deck / Pool Building             106
Campaign / Battle Card Driven     98
Grid Movement                     97
Set Collection                    91
Betting/Wagering                  75
Modular Board                     70
Partnerships                      68
Pattern Building                  56
Area Enclosure                    50
Tile Placement                    50
Chit-Pull System                  49
Memory                            48
Commodity Speculation             40
Acting                            39
Route/Network Building            34
Trick-taking                      33
P

In [10]:
# Look at new category_clean counts.
df_top5000.category_clean.value_counts()

Card Game                  1004
Economic                    333
Adventure                   314
Bluffing                    270
Abstract Strategy           270
                           ... 
Zombies                       2
World War II                  1
Religious                     1
Math                          1
Expansion for Base-game       1
Name: category_clean, Length: 78, dtype: int64

In [11]:
# Look at new designer_clean counts.
df_top5000.designer_clean.value_counts().head(20)

Reiner Knizia        118
(Uncredited)          64
Martin Wallace        51
Bruno Cathala         40
Alan R. Moon          39
Klaus Teuber          39
Richard H. Berg       36
Friedemann Friese     36
Dean Essig            35
Michael Schacht       35
none                  35
Uwe Rosenberg         33
Michael Kiesling      32
Wolfgang Kramer       29
Stefan Feld           27
James Ernest          24
Stefan Dorra          23
Mike Elliott          23
Inka Brand            23
Vlaada Chvátil        23
Name: designer_clean, dtype: int64

In [12]:
# Function to clean the name column.
def clean_name(name):
    # Remove special characters and diacritics
    name = unicodedata.normalize('NFD', name).encode('ascii', 'ignore').decode('utf-8')
    # Replace any remaining special characters with the correct letter
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    return name

In [13]:
# Incorporate the cleaned names into the top 5000 dataset.
df_top5000['name_clean'] = df_top5000['name'].apply(clean_name)
df_top5000.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle


In [14]:
# Top 5000 dataset snapshot.
df_top5000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            5000 non-null   int64  
 1   bgg_url         5000 non-null   object 
 2   id              5000 non-null   int64  
 3   name            5000 non-null   object 
 4   minplayers      5000 non-null   int64  
 5   maxplayers      5000 non-null   int64  
 6   playingtime     5000 non-null   int64  
 7   minplaytime     5000 non-null   int64  
 8   maxplaytime     5000 non-null   int64  
 9   yearpublished   5000 non-null   int64  
 10  average_rating  5000 non-null   float64
 11  geek_rating     5000 non-null   float64
 12  users_rated     5000 non-null   int64  
 13  image_url       5000 non-null   object 
 14  thumb_url       5000 non-null   object 
 15  age             5000 non-null   int64  
 16  mechanic        5000 non-null   object 
 17  total_owners    5000 non-null   i

In [15]:
# Top 5000 dataset snapshot.
df_top5000['name_clean'] = df_top5000.name_clean.replace('',np.nan,regex = True)
df_top5000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            5000 non-null   int64  
 1   bgg_url         5000 non-null   object 
 2   id              5000 non-null   int64  
 3   name            5000 non-null   object 
 4   minplayers      5000 non-null   int64  
 5   maxplayers      5000 non-null   int64  
 6   playingtime     5000 non-null   int64  
 7   minplaytime     5000 non-null   int64  
 8   maxplaytime     5000 non-null   int64  
 9   yearpublished   5000 non-null   int64  
 10  average_rating  5000 non-null   float64
 11  geek_rating     5000 non-null   float64
 12  users_rated     5000 non-null   int64  
 13  image_url       5000 non-null   object 
 14  thumb_url       5000 non-null   object 
 15  age             5000 non-null   int64  
 16  mechanic        5000 non-null   object 
 17  total_owners    5000 non-null   i

In [16]:
# Merge the top 5000 and games datasets.
df = pd.merge(df_top5000, df_games, on=["id"], how="left")
df.head()

,rank,bgg_url,id,name_x,minplayers_x,maxplayers_x,playingtime_x,minplaytime_x,maxplaytime_x,yearpublished_x,average_rating_x,geek_rating,users_rated_x,image_url,thumb_url,age,mechanic,total_owners_x,category,designer,publisher,total_weights_x,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,name_y,yearpublished_y,minplayers_y,maxplayers_y,playingtime_y,minplaytime_y,maxplaytime_y,minage,users_rated_y,average_rating_y,bayes_average_rating,total_owners_y,total_traders,total_wanters,total_wishers,total_comments,total_weights_y,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,Gloomhaven,2016.0,1.0,4.0,120.0,60.0,120.0,12.0,7.0,9.78571,0.00000,42.0,0.0,62.0,476.0,39.0,5.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season,boardgame,Pandemic Legacy,2015.0,2.0,4.0,60.0,60.0,60.0,13.0,12.0,7.33333,0.00000,64.0,0.0,277.0,1853.0,50.0,3.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,Through the Ages: A New Story of Civilization,2015.0,2.0,4.0,240.0,120.0,240.0,14.0,12.0,8.91667,0.00000,9.0,0.0,84.0,686.0,23.0,5.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars,boardgame,Terraforming Mars,2016.0,1.0,5.0,180.0,180.0,180.0,12.0,14.0,8.35714,0.00000,5.0,0.0,37.0,224.0,22.0,6.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle,boardgame,Twilight Struggle,2005.0,2.0,2.0,180.0,180.0,180.0,13.0,20113.0,8.33774,8.22186,26647.0,372.0,1219.0,5865.0,5347.0,2562.0,3.4785


In [17]:
# Columns in the games dataset that are not in the top 5000 dataset.
[x for x in df_games.columns if x not in df_top5000.columns]

['type',
 'minage',
 'bayes_average_rating',
 'total_traders',
 'total_wanters',
 'total_wishers',
 'total_comments',
 'average_weight']

In [18]:
# Create a new dataframe with selected columns.
cols = ['id',
        'type',
 'minage',
 'bayes_average_rating',
 'total_traders',
 'total_wanters',
 'total_wishers',
 'total_comments',
 'average_weight']
df_games_sub = df_games.loc[:, cols]
df_games_sub.head()

,id,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,12333,boardgame,13.0,8.22186,372,1219,5865,5347,3.4785
1,120677,boardgame,12.0,8.14232,132,1586,6277,2526,3.8939
2,102794,boardgame,12.0,8.06886,99,1476,5600,1700,3.7761
3,25613,boardgame,12.0,8.05804,362,1084,5075,3378,4.1590
4,3076,boardgame,12.0,8.04524,795,861,5414,9173,3.2943


In [19]:
# Re-merge
df = pd.merge(df_top5000, df_games_sub, on=["id"], how="left")
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [20]:
# Complete dataset snapshot.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  5000 non-null   int64  
 1   bgg_url               5000 non-null   object 
 2   id                    5000 non-null   int64  
 3   name                  5000 non-null   object 
 4   minplayers            5000 non-null   int64  
 5   maxplayers            5000 non-null   int64  
 6   playingtime           5000 non-null   int64  
 7   minplaytime           5000 non-null   int64  
 8   maxplaytime           5000 non-null   int64  
 9   yearpublished         5000 non-null   int64  
 10  average_rating        5000 non-null   float64
 11  geek_rating           5000 non-null   float64
 12  users_rated           5000 non-null   int64  
 13  image_url             5000 non-null   object 
 14  thumb_url             5000 non-null   object 
 15  age                  

In [ ]:
# Read combined dataset to new csv
df.to_csv("board.csv", index=False)